# Part I. ETL Pipeline for Pre-Processing the Files

#### 1.1 Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### 1.2 Creating list of filepaths to process original event csv data files

In [2]:
# checking the current working directory
print(os.getcwd())

# Get the current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### 1.3 Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below for getting the total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below for checking what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in the new csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion. 

#### Work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### 2.1 Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### 2.2 Create Keyspace

In [6]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### 2.3 Set Keyspace

In [7]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### 2.4 Create queries for the ETL process. Remember to model the database tables on the queries you want to run with Apache Cassandra.

#### 2.4.1 Query 1
##### Create queries to ask the following question:
1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

##### Explanation: 

 - Query 1 wants to extract information based on sessionId and itemInSession, so it is necessary to include both column as the PRIMARY KEY. At the same time, artist, song title and song's length are needed in the output of the query, so these columns need to be included when creating our table.

In [8]:
#CREATE TABLE for query 1
query = "CREATE TABLE IF NOT EXISTS song_library"
query = query + "(sessionId int, itemInSession int, artist text, length float, song text, \
        PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
#INSERT rows for query 1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_library (sessionId, itemInSession, artist, length, song)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], float(line[5]),line[9]))

In [10]:
# Add in the SELECT statement to verify the data was entered into the table
query = "select artist, length, song from song_library where sessionId = 338 and itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.length, row.song)

Faithless 495.30731201171875 Music Matters (Mark Knight Dub)


#### 2.4.2 Query 2
##### Create queries to ask the following question:
2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

##### Explanation: 

 - Query 2 wants to extract information based on userId and sessionId, and also requies the query to be sorted by itemInSession. Hence it is necessary to include all these column in the PRIMARY KEY bracket.
 - itemInsession is determining the sort and will not be used in the where clause, so we can include it as a CLUSTERING COLUMN. 
 - On the other hand, sessions belong to the same use may distributed into different node, so there can be a performance issue because the data is large. In this case, we select both userId and sessionId as the PARTITION KEYs, so that sessions from the same user will be stored together.  
 - Last but not least, we should include artist, song and users' first and last name as well since they are needed in the output of the query.

In [11]:
#CREATE TABLE for query 2
query = "CREATE TABLE IF NOT EXISTS log_library"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, \
        PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [12]:
#INSERT rows for query 2
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO log_library (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [13]:
# Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, LastName from log_library where userId = 10 and sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


#### 2.4.3 Query 3
##### Create queries to ask the following question:
3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

##### Explanation: 

 - Query 3 wants to extract information based on the song, but different user can listen to the same song, or moreover, same user can listen to the same song repeatedly as well. This question wants to get song and user information. How can we differenciate that since song or even song + user is not unique?
 - Performed some searching and quering on the dataset, found the fact that song + user + sessionId has duplicates, and song+ user+itemInSession has duplicates as well. However, song + user + sessionId + itemInSession does not have duplicate, as this combination provides all unique entries.
 - Because of this, song is selected as the PARTITION KEY, and userId, sessionId and itemInSession as the nested CLUSTERING COLUMNS)
 - It would be better to understand the meaning of the sessionId and itemInSession fields, so that we could more easily to understand how to design the database. Since I did not have that information, I just simply looked at the data and manually checking what is the unique combination with the least number of columns.

In [14]:
#CREATE TABLE for query 3
query = "CREATE TABLE IF NOT EXISTS listen_library"
query = query + "(song text, userId int, sessionId int, itemInSession int, firstName text, \
        lastName text, PRIMARY KEY(song, userId, sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e) 

In [15]:
#INSERT rows for query 3
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO listen_library (song, userId, sessionId, itemInSession, firstName, lastName)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s)"
        session.execute(query, (line[9], int(line[10]), int(line[8]), int(line[3]), line[1], line[4]))

In [16]:
# Add in the SELECT statement to verify the data was entered into the table
query = "select firstName, LastName from listen_library where song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### 2.5 Drop the tables before closing out the sessions

In [17]:
query = "drop table song_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table log_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "drop table listen_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### 2.6 Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()